In [7]:
search_for = 93
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002324342727661133
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 115470905
type: [1, 1, 1, 1, 93] 93
cases of this type: 74805201
10000 13.747394704901774
20000 15.07410518638134
30000 30.585064394356078
40000 52.41152601209932
50000 53.17861537567943
60000 52.61168508873284
70000 53.886124640884006
80000 58.45331031890074
90000 62.286839138566194
100000 65.17120737561073
110000 70.07641972130577
120000 70.42432083657212
130000 68.1427383502251
140000 67.44809548157932
150000 67.69589453478045
160000 70.01161144581636
170000 69.86101033369462
180000 66.14326387823691
190000 70.45845629690737
200000 72.75891457969755
210000 77.86110870393335
220000 72.72480512113948
230000 68.3802623175609
240000 68.4609554173197
250000 69.45385495905214
260000 66.9222828475976
270000 66.24388816879085
280000 64.18246522296876
290000 65.9038295350349
300000 67.51990722918013
310000 69.92969948981643
320000 66.

3110000 59.637992937189644
3120000 61.74840767979132
3130000 69.28505254989723
3140000 57.709995391215976
3150000 58.55685833650702
3160000 71.33613924377329
3170000 46.35537271781136
3180000 23.719756678322952
3190000 23.37854612933735
3200000 23.64324717233986
3210000 29.119302971554035
3220000 83.77117274648182
3230000 77.44710578617642
3240000 51.87297331917485
3250000 19.379561951854395
3260000 19.43791224345098
3270000 19.23397974587437
3280000 29.81445633022831
3290000 74.80197733480573
3300000 65.09193894499735
3310000 61.37423896351434
3320000 66.53212317994515
3330000 62.93651747733017
3340000 57.053537821443975
3350000 60.08019727644802
3360000 64.3363941637469
3370000 58.33314389256093
3380000 59.75033610820144
3390000 66.6411915710517
3400000 62.80188424928635
3410000 67.06235043882639
3420000 66.69952013626109
3430000 62.13428791242861
3440000 71.54160617927045
3450000 65.97916607870512
3460000 59.2207211753877
3470000 64.08636230028331
3480000 58.66579668485588
3490000 5

6250000 55.50016974350618
6260000 54.73843894933193
6270000 57.51115327654594
6280000 59.435742955322425
6290000 56.84509994361609
6300000 59.203135863127244
6310000 54.944706442585264
6320000 59.05150727242777
6330000 57.17903132568813
6340000 57.94304713152634
6350000 57.12935012686908
6360000 23.341320998236736
6370000 23.405049130350882
6380000 22.704472557367982
6390000 22.87062065169262
6400000 64.29690446530097
6410000 59.375308132753766
6420000 57.02448099564039
6430000 60.67323958811081
6440000 61.05341792009681
6450000 66.60534643831551
6460000 60.23051286042597
6470000 53.78349417042653
6480000 64.63639752280908
6490000 55.720720670204365
6500000 18.38645374818941
6510000 19.05091007996112
6520000 18.86801569887343
6530000 18.875645203424096
6540000 65.28446288720369
6550000 62.86710155870871
6560000 59.24712224156529
6570000 56.19661509674217
6580000 60.48810489439805
6590000 61.49537026306461
6600000 62.70758438449883
6610000 58.85065243377113
6620000 58.42205738930673
663

9380000 52.593908165761725
9390000 55.11496707772693
9400000 56.52628068110893
9410000 56.30621530567914
9420000 57.1838912679806
9430000 53.71915835039355
9440000 57.59946174765894
9450000 57.34885313378443
9460000 56.426076281750305
9470000 57.18922764159739
9480000 60.61196056474418
9490000 54.056107933220964
9500000 61.35142886481732
9510000 56.81852148071475
9520000 55.44730813366618
9530000 59.08843871940394
9540000 27.195909252624844
9550000 22.303905713435213
9560000 22.49322108196189
9570000 22.26056154016687
9580000 46.63153094075146
9590000 63.10261375260473
9600000 53.355725686947935
9610000 55.76742484936826
9620000 57.40812064015666
9630000 58.061384681545306
9640000 59.958223411121374
9650000 61.32782163189729
9660000 62.21276311020285
9670000 60.636503049711486
9680000 56.499650726488234
9690000 60.30249408399562
9700000 55.48302143377771
9710000 58.8365878030463
9720000 57.784903053857285
9730000 57.38607227040418
9740000 61.633221596295826
9750000 18.093542188251053
9

12410000 48.750926948031385
12420000 50.33867649330342
12430000 46.85197270859665
12440000 51.49886738058746
12450000 37.26626497704761
12460000 9.051399217900899
12470000 8.98118291470051
12480000 8.908249701504012
12490000 15.260930081858866
12500000 58.646713175666136
12510000 51.010750943693914
12520000 44.27254306132546
12530000 50.197201304277186
12540000 49.38003253076924
12550000 49.213395112507904
12560000 45.25082875040978
12570000 47.67743995205565
12580000 47.079107578572064
12590000 47.73199837140888
12600000 52.25041424070004
12610000 55.55395114454008
12620000 55.656501692504875
12630000 50.95538013938186
12640000 50.99054437814968
12650000 48.84704082556636
12660000 49.49313967952735
12670000 52.67777608979666
12680000 55.94161690593929
12690000 50.679282841218054
12700000 52.03682133271721
12710000 53.90063919392715
12720000 38.58217581566844
12730000 21.585666317218113
12740000 22.117283865424394
12750000 21.993169242575895
12760000 34.41765442870041
12770000 63.67326

...

102350000 0.6000290839478374
102360000 0.787554585660994
102370000 0.827417268795669
102380000 0.8646098816204071
102390000 0.8587505779475967
102400000 0.8648207203619679
102410000 0.8755727061082919
102420000 0.802758573097951
102430000 0.8630352266678547
102440000 0.8751018631404639
102450000 0.8309528228725989
102460000 0.7554460217485164
102470000 0.8032792882096769
102480000 0.8584912345109053
102490000 0.7299984143701196
102500000 0.8573830228275061
102510000 0.8481379206568996
102520000 0.8086071368896961
102530000 0.7671484801503023
102540000 0.6540778657060862
102550000 0.8710471381253998
102560000 0.7191007795661687
102570000 0.7930141697614392
102580000 0.8492776725914082
102590000 0.5683965570557118
102600000 0.8160441500106453
102610000 0.8463033965648213
102620000 0.8547789928920071
102630000 0.8514655506047937
102640000 0.8370619722466005
102650000 0.8158652300551533
102660000 0.6119937753706177
102670000 0.6590062123578124
102680000 0.6509389225935936
102690000 0.85035

105200000 2.2008621345673003
105210000 2.348299309705198
105220000 1.9918825721309583
105230000 2.2211529100662473
105240000 1.878295523461236
105250000 2.7497667072445484
105260000 2.445177407743037
105270000 2.1466996603505484
105280000 2.26088755037821
105290000 2.3903522388925156
105300000 2.3884025214202205
105310000 2.67051252510945
105320000 2.7473639109161496
105330000 1.9627846941583027
105340000 2.1817165457649694
105350000 2.155599667981565
105360000 2.49870570870926
105370000 2.550311650665237
105380000 2.5671461636348565
105390000 1.916522456117206
105400000 1.8527447364941239
105410000 2.3356574025727315
105420000 2.383592158692148
105430000 2.3867668618174394
105440000 1.520381901744604
105450000 2.6506858174708157
105460000 2.538740128617651
105470000 2.168997637183269
105480000 1.988465029033323
105490000 2.0955804067227906
105500000 2.5270907050244014
105510000 2.840910992995269
105520000 2.637645937154558
105530000 2.1032969266849757
105540000 1.7963969712405734
1055

108050000 2.101769894382656
108060000 1.765139131601486
108070000 1.4302131803715892
108080000 1.7413096602758766
108090000 1.8032739319992066
108100000 1.500244475072821
108110000 1.458945464397371
108120000 1.5435907247509557
108130000 1.725259116859701
108140000 1.8469713550767302
108150000 1.9725500389671655
108160000 1.4905088584937984
108170000 1.5291854304405053
108180000 1.8144476088744073
108190000 1.9950401230020987
108200000 2.004283984096249
108210000 1.7636670010596514
108220000 1.7382673346645963
108230000 1.6913954236087203
108240000 1.4555642131291497
108250000 1.8642688807204366
108260000 1.6216855471663671
108270000 1.7898738632631965
108280000 1.6339325223773054
108290000 1.5113372608390452
108300000 1.614100481285751
108310000 1.3592030635037355
108320000 1.8062363006964326
108330000 1.4913225800488392
108340000 1.4490032197363842
108350000 0.8911504539477825
108360000 1.2657109014550514
108370000 1.4885479903557899
108380000 1.6534820684105158
108390000 1.617493799

110900000 0.7202488732162118
110910000 0.9122745228272344
110920000 0.8660866247891387
110930000 1.142047128202021
110940000 1.1348951558237605
110950000 1.0548048497105968
110960000 1.0980071439000962
110970000 0.8801729488857918
110980000 0.6038867869249649
110990000 0.7736654296825329
111000000 1.3489614712958535
111010000 1.2818542825164065
111020000 0.8773809240961075
111030000 1.3449652902683946
111040000 0.8649603978474936
111050000 0.8358614434442918
111060000 1.083915242741969
111070000 0.8693018375577197
111080000 0.6490427669424812
111090000 0.7818788391038113
111100000 1.176585593984392
111110000 0.9457805577367544
111120000 1.2990762031700878
111130000 1.1223533562591672
111140000 1.2278028014547624
111150000 1.4827906340544093
111160000 1.3259957562967142
111170000 0.9020704601669313
111180000 1.0422964914238122
111190000 0.7333053921491901
111200000 0.7271649877476692
111210000 0.9817136380001903
111220000 0.7596735895735025
111230000 0.9986561058172585
111240000 1.07982

113600000 0.3828382795410355
113610000 0.49770862102094626
113620000 0.6889271139082643
113630000 0.526782278098762
113640000 0.2326834786392914
113650000 0.2785634634774261
113660000 0.28068212781339885
113670000 0.2877552900868323
113680000 0.4465655267905196
113690000 0.3458069305505355
113700000 0.5877361727238695
113710000 0.5527425292961133
113720000 0.44853269889503716
113730000 0.22790670306242172
113740000 0.40582301614089145
113750000 0.2906627003566424
113760000 0.3000903894962867
113770000 0.30613042491896286
113780000 0.27233203399936357
113790000 0.5543538690328598
113800000 0.610580042000148
113810000 0.2922360433897376
113820000 0.21942743609862195
113830000 0.5124500671188368
113840000 0.24487914533933003
113850000 0.2873352881702118
113860000 0.31136792566733223
113870000 0.2634431669686238
113880000 0.4601076500435339
113890000 0.5084666757670707
113900000 0.2612540255984664
113910000 0.34932254330962903
113920000 0.46207950532611874
113930000 0.19643918756405512
113